In [97]:
import numpy as np
import pandas as pd
import psycopg2

file_path = "reports_from_branches/chesterfield_25-08-2021_09-00-00.csv"
column_names = ["date_time", "branch", "customer", "order_content", "total_price", "payment_type", "credit_card_number"]
df = pd.read_csv(file_path, names=column_names)

In [98]:
def cleaning_and_arranging_df(df):
     
    #spliting the first time, turn order_content into a list
    df["order_content"]= df["order_content"].str.split(',')

    #spliting the order_content into individual row
    df = df.explode("order_content")

    #spliting from the last -, turn it into a list
    df["order_content"] = df["order_content"].str.rsplit('-', 1)

    #putting the two list items into two newly created columns
    df[['basket_item','item_price']] = pd.DataFrame(df.order_content.tolist(), index= df.index)

    #striping whitespaces
    df['item_price'] = df['item_price'].str.strip()
    df['basket_item'] = df['basket_item'].str.strip()

    #drop old columns
    df = df.drop(columns=["order_content"])

    #rearrange columns
    new_col = ["date_time", "branch", "customer", "basket_item","item_price", "total_price", "payment_type","credit_card_number"]
    df = df[new_col]

    return df

In [99]:
def dropping_colums(df,drop_cols):
    df = df.drop(drop_cols, axis=1)
    return df

In [100]:
df = cleaning_and_arranging_df(df)
df

/var/folders/vp/zs3dkrsd34nd2b1kk31fvwwh0000gn/T/ipykernel_9315/2050942264.py:10: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df["order_content"] = df["order_content"].str.rsplit('-', 1)


,date_time,branch,customer,basket_item,item_price,total_price,payment_type,credit_card_number
0,25/08/2021 09:00,Chesterfield,Richard Copeland,Regular Flavoured iced latte - Hazelnut,2.75,5.20,CARD,5.494174e+15
0,25/08/2021 09:00,Chesterfield,Richard Copeland,Large Latte,2.45,5.20,CARD,5.494174e+15
1,25/08/2021 09:02,Chesterfield,Scott Owens,Large Flavoured iced latte - Caramel,3.25,17.30,CARD,6.844802e+15
1,25/08/2021 09:02,Chesterfield,Scott Owens,Regular Flavoured iced latte - Hazelnut,2.75,17.30,CARD,6.844802e+15
1,25/08/2021 09:02,Chesterfield,Scott Owens,Regular Flavoured iced latte - Caramel,2.75,17.30,CARD,6.844802e+15
...,...,...,...,...,...,...,...,...
265,25/08/2021 16:52,Chesterfield,Sharon Campanelli,Regular Flavoured iced latte - Caramel,2.75,10.70,CASH,NaN
265,25/08/2021 16:52,Chesterfield,Sharon Campanelli,Regular Flavoured latte - Hazelnut,2.55,10.70,CASH,NaN
265,25/08/2021 16:52,Chesterfield,Sharon Campanelli,Regular Flat white,2.15,10.70,CASH,NaN
266,25/08/2021 16:56,Chesterfield,Henry Frew,Regular Flavoured latte - Hazelnut,2.55,2.55,CASH,NaN


In [101]:
drop_for_product = ["date_time", "branch", "customer", "total_price", "payment_type","credit_card_number"]
drop_for_basket = ["date_time", "branch", "customer", "basket_item","item_price", "total_price", "payment_type","credit_card_number"]
drop_for_transaction = ["customer", "basket_item","item_price","credit_card_number"]

In [102]:
transaction_df = dropping_colums(df, drop_for_transaction)
transaction_df["date_time"] = pd.to_datetime(transaction_df["date_time"])
transaction_df = transaction_df.drop_duplicates()
transaction_df

,date_time,branch,total_price,payment_type
0,2021-08-25 09:00:00,Chesterfield,5.20,CARD
1,2021-08-25 09:02:00,Chesterfield,17.30,CARD
2,2021-08-25 09:04:00,Chesterfield,4.60,CARD
3,2021-08-25 09:06:00,Chesterfield,5.00,CARD
4,2021-08-25 09:08:00,Chesterfield,4.60,CASH
...,...,...,...,...
263,2021-08-25 16:45:00,Chesterfield,14.35,CARD
264,2021-08-25 16:49:00,Chesterfield,6.10,CASH
265,2021-08-25 16:52:00,Chesterfield,10.70,CASH
266,2021-08-25 16:56:00,Chesterfield,2.55,CASH


In [103]:
product_df = dropping_colums(df, drop_for_product)

product_df.drop_duplicates()


,basket_item,item_price
0,Regular Flavoured iced latte - Hazelnut,2.75
0,Large Latte,2.45
1,Large Flavoured iced latte - Caramel,3.25
1,Regular Flavoured iced latte - Caramel,2.75
1,Large Flavoured iced latte - Hazelnut,3.25
1,Regular Flavoured latte - Hazelnut,2.55
2,Large Flat white,2.45
2,Regular Latte,2.15
5,Regular Flat white,2.15
6,Large Flavoured latte - Hazelnut,2.85


In [ ]:
basket_df = dropping_colums(df, drop_for_basket)

In [ ]:
transaction_df

In [ ]:
basket_df['basket_id'] = basket_df.index

In [ ]:
basket_df